### Connect and authorize


In [4]:
import ee

try:
  ee.Initialize()
  print('The Earth Engine package initialized successfully!')
except ee.EEException as e:
  print('The Earth Engine package failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

The Earth Engine package initialized successfully!


#### On authorization failure

In [ ]:
%%bash
earthengine authenticate --quiet

In [ ]:
%%bash
earthengine authenticate --authorization-code=4/c5Ww2rNPGnbNhq0UuuJDxOiZwNKXX2W80v31BC8m7AM

### Import Packages

In [2]:
import urllib
import zipfile
import sys, os
import time
from sys import stdout
from PIL import Image, ImageMath
from datetime import date, timedelta

### Define Functions for Requests

In [2]:
def ImportTifImage(image_list):
    # Create range for request
    geometry = ee.Geometry.Rectangle([68.18625, 6.754256, 97.41516, 35.50133])
    
    # Get download paths for images
    for date in image_list.keys():
        # avg_rad
        img_name = image_list[date][0].getDownloadUrl({
            'scale': 1024,
            'region': geometry['coordinates'][0]
        })

        stdout.write("\nDownloading from... \n...\r%s" % img_name)

        # Download file
        t_filename = str(date) + "_tif_file"
        file_dwnldr = urllib.URLopener()
        file_dwnldr.retrieve(img_name, t_filename)
        
        # Storage for filenames to be transferred
        transfers = []
        
        # Unzip and transform
        with zipfile.ZipFile(t_filename, 'r') as zipped_img:
            stdout.write("\nTransforming...\r%s" % zipped_img.namelist()[1])
            stdout.flush()
            # Load tif file 
            img = Image.open(zipped_img.extract(zipped_img.namelist()[1]))
            # Gamma correction
            img = ImageMath.eval("a**(gamma)", a=img, gamma=10)
            # Save 32 bit int / pixel
            img.convert('I').save("32bit/" + str(zipped_img.namelist()[1]))
            # Save 8 bit int / pixel
            img.convert('L').save("8bit/" + str(zipped_img.namelist()[1]))
            # Remove zipped file
            os.remove(zipped_img.namelist()[1])
            # Store for transfer
            transfers.append(str(zipped_img.namelist()[1]))
        # Remove downloaded file
        os.remove(t_filename)
    stdout.write("\nComplete.\n")
    
    # Return transfers list
    return transfers

### Pull Image Collection

#### Download image files, unzip, transform, and store.

##### All Image Files

In [ ]:
# All Image Files
## Create dates List
dates = list(range(20140101, 20141301, 100)) + \
        list(range(20150101, 20151301, 100)) + \
        list(range(20160101, 20161301, 100)) + \
        list(range(20170101, 20171001, 100))
## Create images list for avg_rad, cf_avg
image_list = {}
for date in dates:
  ## Create tuple of avg_rad, cf_avg
  image_list[date] = ((
    ee.Image("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG/" + str(date)).select('avg_rad'), 
    ee.Image("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG/" + str(date)).select('cf_avg')
  ))
  
# Import
transfers = ImportTifImage(image_list)
# Clear
image_list = None

##### Most Recent Image File Only

In [27]:
# Only most recent file (3 months ago due to earth engine delays)
date_min = date.today() - timedelta(days=date.today().day + 1)
date_min = date_min - timedelta(days=date_min.day + 1)
date_min = date_min - timedelta(days=date_min.day + 1)
date_min = date_min - timedelta(days=date_min.day - 1 )
date_min = str(date_min).replace('-', '')

## Create tuple of avg_rad, cf_avg
image_list = {}
image_list[date_min] = ((
  ee.Image("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG/" + str(date_min)).select('avg_rad'), 
  ee.Image("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG/" + str(date_min)).select('cf_avg')
))

# Import
transfers = ImportTifImage(image_list)
# Clear
image_list = None


https://earthengine.googleapis.com/api/download?docid=2b064a694192b9f94b288c68a3c107bc&token=ad92e671caf305e8aa3ebe9711b038b5
20170901.avg_rad.tif
Complete.

### Transfer Downloaded Image Files to GCP Bucket

In [28]:
for filename in transfers:
    stdout.write("\nTransferring files to GCP...\n")
    # Transfer 32bit
    tsf_32 = '32bit/' + filename
    !gsutil cp '$tsf_32' gs://soli_ee_data/earthengine/VIIRS/india_32bit
    # Transfer 8bit
    tsf_8 = '8bit/' + filename
    !gsutil cp '$tsf_8' gs://soli_ee_data/earthengine/VIIRS/india_8bit

stdout.write("\nComplete.\n")
stdout.flush()


Transferring files to GCP...
Copying file://32bit/20170901.avg_rad.tif [Content-Type=image/tiff]...
| [1 files][ 37.9 MiB/ 37.9 MiB]                                                
Operation completed over 1 objects/37.9 MiB.                                     
Copying file://8bit/20170901.avg_rad.tif [Content-Type=image/tiff]...
- [1 files][  9.5 MiB/  9.5 MiB]                                                
Operation completed over 1 objects/9.5 MiB.                                      

Complete.


### Transfer from GCP Bucket to VMs Bucket

In [4]:
%%bash
gsutil cp gs://soli_ee_data/earthengine/Transformed_assets/ gs://peerless-robot-187203/india_viirs_8bit.tif

Copying gs://peerless-robot-187203/india_viirs_8bit.tif [Content-Type=image/tiff]...
- [1 files][  2.8 MiB/  2.8 MiB]                                                
Operation completed over 1 objects/2.8 MiB.                                      


### Upload to Earth Engine

In [20]:
## TODO: Loop through files in directory
asset_id = "full_test"
filename = "mpce.tif"

!earthengine upload image --asset_id=users/nvogler/soli/'$asset_id' gs://soli_ee_data/earthengine/Transformed_assets/'$filename'

Started upload task with ID: 3NNB2XYZQYCQAIX3G2ORF2VQ


In [31]:
import google.datalab.storage as storage
filenames = [o.key for o in storage.Bucket('soli_ee_data').objects()
  if o.key.startswith('earthengine/Transformed_assets/')]

for filename in filenames:
  # Asset ID, Filename
  xfer_file = filename.split('/')[-1]
  asset_id = xfer_file.split('.')[0]
  # Verify not null
  if asset_id != "":
    print ("Asset ID: " + asset_id)
    print ("Filename: " + xfer_file)
    # Upload to Earth Engine
    try:
      !earthengine upload image --asset_id=users/nvogler/soli/'$asset_id' gs://soli_ee_data/earthengine/Transformed_assets/'$xfer_file'
    except(e):
      print ("Error uploading image to Earth Engine.\n" + str(e))

Asset ID: Median MPCE
Filename: Median MPCE.tiff
Invalid asset id. Only letters, numbers, underscores, hyphens, and slashes are allowed.
Asset ID: india_viirs_8bit
Filename: india_viirs_8bit.tif
Started upload task with ID: ZBF5TIFA4PWEEODIMEXOYV6N
Asset ID: mpce
Filename: mpce.tif
Started upload task with ID: S4LBDNQANXOWTQEF7DZR3WVB
